In [14]:
import numpy as np

# ユーザ定義関数

In [15]:
# 前処理をまとめた関数：corpus, word_to_id, id_to_word を出力
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

In [16]:
# 共起行列を得る関数
def create_co_matrix(corpus, vocab_size, window_size=1):
    '''共起行列の作成

    :param corpus: コーパス（単語IDのリスト）
    :param vocab_size:語彙数
    :param window_size:ウィンドウサイズ（ウィンドウサイズが1のときは、単語の左右1単語がコンテキスト）
    :return: 共起行列
    '''
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1

    return co_matrix

In [17]:
# コサイン類似度を出力する関数
def cos_similality(x, y, eps=1e-8):
    nx = x / (np.sqrt(np.sum(x**2)) + eps)
    ny = y / (np.sqrt(np.sum(y**2)) + eps)
    return np.dot(nx, ny)

In [41]:
# ある単語クエリに対して類似した単語を上位から順に表示する関数
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    # 1_クエリを取り出す
    if query not in word_to_id:
        print(f'{query} is not found') # f-string
        return
    print('\n[query]' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]
    # 2_コサイン類似度の算出
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similality(word_matrix[i], query_vec)
    # コサイン類似度の結果からその値を高い順に出力
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(f' {id_to_word[i]}: {similarity[i]}')
        
        count += 1
        if count >= top:
            return

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#2.3_カウントベースの手法" data-toc-modified-id="2.3_カウントベースの手法-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>2.3_カウントベースの手法</a></span><ul class="toc-item"><li><span><a href="#2.3.4_共起行列" data-toc-modified-id="2.3.4_共起行列-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>2.3.4_共起行列</a></span></li><li><span><a href="#2.3.5_ベクトル間の類似度" data-toc-modified-id="2.3.5_ベクトル間の類似度-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>2.3.5_ベクトル間の類似度</a></span></li><li><span><a href="#2.3.6_類似単語のランキング表示" data-toc-modified-id="2.3.6_類似単語のランキング表示-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>2.3.6_類似単語のランキング表示</a></span></li></ul></li></ul></div>

## 2.3_カウントベースの手法

In [18]:
text = 'You say goodby and I say hello.'

In [19]:
text = text.lower()
text = text.replace('.', ' .')
text

'you say goodby and i say hello .'

In [20]:
words = text.split(' ')
words

['you', 'say', 'goodby', 'and', 'i', 'say', 'hello', '.']

In [21]:
# 単語IDから単語、単語から単語IDへの変換を可能にする辞書
word_to_id = {}
id_to_word = {}

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

In [22]:
print(word_to_id)

{'you': 0, 'say': 1, 'goodby': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}


In [23]:
print(id_to_word)

{0: 'you', 1: 'say', 2: 'goodby', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [24]:
word_to_id['say']

1

In [25]:
id_to_word[4]

'i'

In [26]:
# 単語のリストを単語IDのリストに変換
corpus = [word_to_id[x] for x in words] 
corpus = np.array(corpus)
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

In [27]:
text = 'You say goodby and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

### 2.3.4_共起行列

In [28]:
text = 'You say goodby and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

In [29]:
print(corpus)

[0 1 2 3 4 1 5 6]


In [30]:
print(id_to_word)

{0: 'you', 1: 'say', 2: 'goodby', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [31]:
# 共起行列を計算
vocab_size = len(word_to_id) # 語彙数を指定
C = create_co_matrix(corpus, vocab_size, window_size=1)
C

array([[0, 1, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 1, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0]], dtype=int32)

### 2.3.5_ベクトル間の類似度

cosine similality
similality(x, y) = x・y / (||x|| ||y||)
2つのベクトルがどれだけ同じ方向を向いているかを表す。
一致=1,　逆方向=-1

In [33]:
text = 'You say goodby and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

In [34]:
c0 = C[word_to_id['you']] # 'you'の単語ベクトル
c1 = C[word_to_id['i']] # 'i'の単語ベクトル
print(cos_similality(c0, c1))

0.7071067691154799


### 2.3.6_類似単語のランキング表示

In [42]:
text = 'You say goodby and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

most_similar('you', word_to_id, id_to_word, C, top=5)


[query]you
 goodby: 0.7071067691154799
 i: 0.7071067691154799
 hello: 0.7071067691154799
 say: 0.0
 and: 0.0


In [ ]:
test